# 教程 4: CANN 参数效应

> **阅读时间**: 约30-35分钟
> **难度**: 中级
> **先决条件**: [教程 1-3](./01_build_cann_model.ipynb)

本教程系统地探索不同 CANN1D 参数如何影响模型动力学。

---

## 目录

1. [参数概述](#1-parameter-overview)
2. [实验设置](#2-experimental-setup)
3. [参数探索](#3-parameter-exploration)
4. [后续步骤](#4-next-steps)

---

## 1. 参数概览

CANN1D具有以下主要参数：

| 参数 | 默认值 | 物理意义 |
|-----------|---------|------------------|
| `num` | 256 | 神经元数量（网络分辨率） |
| `tau` | 1.0 | 时间常数（动力学速度） |
| `k` | 8.1 | 全局抑制强度 |
| `a` | 0.5 | 连接宽度（局部兴奋范围） |
| `A` | 10 | 外部输入振幅 |
| `J0` | 4.0 | 突触连接强度 |
| `tau_v` | 50.0 | SFA时间常数（仅限CANN1D_SFA） |

我们将使用**SmoothTracking1D**任务和**energy_landscape_1d_animation**可视化来观察每个参数的影响。

---

## 2. 实验设置

### 2.1 基础代码框架

In [1]:
import brainstatefrom canns.models.basic import CANN1D, CANN1D_SFAfrom canns.task.tracking import SmoothTracking1Dfrom canns.analyzer.plotting import (    PlotConfigs,    energy_landscape_1d_animation)# 设置环境brainstate.environ.set(dt=0.1)def run_experiment(model, title="", save_path=None):    """运行标准实验并可视化结果"""    model.init_state()    # 创建平滑追踪任务    task = SmoothTracking1D(        cann_instance=model,        Iext=[0.0, 3.0, 2.0, 3.0],        duration=[20.0, 20.0, 10.0],        time_step=0.1,    )    # 获取任务数据    task.get_data()    def run_step(t, inp):        model.update(inp)        return model.u.value, model.r.value, model.inp.value    u_history, r_history, input_history = brainstate.transform.for_loop(        run_step,        task.run_steps,        task.data,        pbar=brainstate.transform.ProgressBar(10)    )    # 配置并创建可视化    config = PlotConfigs.energy_landscape_1d_animation(        time_steps_per_second=100,        fps=20,        title=title,        xlabel='Position',        ylabel='Firing Rate',        repeat=True,        show=True,        save_path=save_path    )    energy_landscape_1d_animation(        data_sets={'u': (model.x, u_history), 'stimulus': (model.x, input_history)},        config=config    )    return r_history

### 2.2 默认参数基线

首先，使用默认参数建立基线：

In [2]:
# 默认参数
model_default = CANN1D(
    num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0
)
r_default = run_experiment(
    model_default,
    title="Default Parameters",
    save_path=None  # 设置为 'default.gif' 来保存
)

<SmoothTracking1D> Generating Task data: 500it [00:00, 997.94it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

**观察要点**：凸起应该平稳地从左到右跟踪刺激，形状稳定且清晰。

---

## 3. 参数探索

对于每个参数，我们将使用多个实验创建并排比较。这种方法允许您直接比较不同的参数值。

### 3.1 神经元数量 `num`

`num` 控制网络分辨率。更多的神经元意味着更精细的特征表示，但计算成本更高。

In [3]:
# 测试不同的神经元数量
for num_val in [64, 256, 512]:
    model = CANN1D(num=num_val, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0)
    run_experiment(
        model,
        title=f"神经元数量: {num_val}",
        save_path=None
    )

<SmoothTracking1D> Generating Task data: 500it [00:00, 1007.50it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 7830.98it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 1487.74it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

**观察**：
- `num=64`：较低分辨率，颠簸表示较粗糙
- `num=256`：平衡分辨率，标准选择
- `num=512`：更高分辨率，颠簸更平滑，但计算量增加

**关键洞察**：分辨率影响颠簸平滑度，但不影响基本动力学。当精确的空间表示很重要时，使用更高的 `num`。

### 3.2 时间常数 `tau`

`tau` 控制动力学时间尺度。较大的 `tau` 意味着对输入的响应较慢。

In [4]:
# 测试不同的时间常数
for tau_val in [0.5, 1.0, 2.0]:
    model = CANN1D(num=256, tau=tau_val, k=8.1, a=0.5, A=10, J0=4.0)
    run_experiment(
        model,
        title=f"Time Constant: tau={tau_val}",
        save_path=None
    )

<SmoothTracking1D> Generating Task data: 500it [00:00, 6681.74it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 7698.46it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 1703.66it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

**观察**：
- `tau=0.5`：快速响应，凸起紧密跟随刺激，延迟最小
- `tau=1.0`：标准响应，平衡的跟踪速度
- `tau=2.0`：慢速响应，凸起滞后于移动的刺激

**关键洞察**：`tau` 控制网络的时间滤波。较小的 `tau` 意味着对变化输入的更快适应，较大的 `tau` 意味着更平滑、更慢的动力学。

### 3.3 全局抑制 `k`

`k` 控制全局抑制强度，对于维持单凸起吸引子状态至关重要。

In [5]:
# 测试不同的抑制强度
for k_val in [4.0, 8.1, 16.0]:
    model = CANN1D(num=256, tau=1.0, k=k_val, a=0.5, A=10, J0=4.0)
    run_experiment(
        model,
        title=f"全局抑制: k={k_val}",
        save_path=None
    )

<SmoothTracking1D> Generating Task data: 500it [00:00, 7221.90it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 4806.87it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 5538.20it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

**观察结果**：
- `k=4.0`：弱抑制，凸起可能不稳定、较宽或分裂成多个凸起
- `k=8.1`：平衡的抑制，稳定的单个凸起
- `k=16.0`：强抑制，更尖锐、更窄的凸起

**关键洞察**：`k` 平衡兴奋和抑制。过弱：不稳定性或多个凸起。过强：凸起可能无法形成或过于狭窄。

### 3.4 连接宽度 `a`

`a` 控制局部兴奋连接的空间范围，直接影响凸起的宽度。

In [6]:
# 测试不同的连接宽度
for a_val in [0.3, 0.5, 0.8]:
    model = CANN1D(num=256, tau=1.0, k=8.1, a=a_val, A=10, J0=4.0)
    run_experiment(
        model,
        title=f"连接宽度: a={a_val}",
        save_path=None
    )

<SmoothTracking1D> Generating Task data: 500it [00:00, 4087.94it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 2955.45it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 7412.03it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

**观察**：
- `a=0.3`：窄连接，产生更窄的凸起，峰值更尖锐
- `a=0.5`：标准宽度，平衡的凸起形状
- `a=0.8`：宽连接，产生更宽、分布更广的凸起

**关键洞察**：`a` 直接控制空间感受野大小。根据应用所需的空间精度调整 `a`。

### 3.5 外部输入振幅 `A`

`A` 控制外部刺激输入的振幅。

In [7]:
# 测试不同的输入幅度
for A_val in [5, 10, 20]:
    model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=A_val, J0=4.0)
    run_experiment(
        model,
        title=f"输入幅度: A={A_val}",
        save_path=None
    )

<SmoothTracking1D> Generating Task data: 500it [00:00, 3244.44it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 4569.55it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 634.16it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

**观察**：
- `A=5`：较弱的输入，凸起的峰值放电率较低
- `A=10`：标准幅度，平衡的响应
- `A=20`：较强的输入，凸起的峰值放电率较高，且可能传播范围更广

**关键洞察**：`A` 控制外部输入驱动网络的强度。较高的 `A` 使凸起对外部刺激相比内部循环动力学更有响应。

### 3.6 突触连接强度 `J0`

`J0` 控制循环突触连接的最大强度。

In [8]:
# 测试不同的连接强度
for J0_val in [2.0, 4.0, 8.0]:
    model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=J0_val)
    run_experiment(
        model,
        title=f"突触强度: J0={J0_val}",
        save_path=None
    )

<SmoothTracking1D> Generating Task data: 500it [00:00, 4871.12it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 5754.28it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 590.77it/s] 


  0%|          | 0/500 [00:00<?, ?it/s]

**观察**：
- `J0=2.0`：弱递归连接，颠簸可能在没有输入的情况下无法保持稳定性
- `J0=4.0`：标准连接，稳定的颠簸维持
- `J0=8.0`：强递归连接，非常稳定的颠簸，可能会变得过于僵硬

**关键洞察**：`J0` 决定了吸引子的强度。更高的 `J0` 意味着更强的自我维持动力学和更好的记忆维持，但可能会降低对输入变化的响应性。

### 3.7 SFA 时间常数 `tau_v`

对于 `CANN1D_SFA`，`tau_v` 控制尖峰频率适应时间尺度。SFA 产生一个适应电流，即使在没有外部速度输入的情况下，也可能导致颠簸漂移。

In [9]:
# 测试不同的SFA时间常数
for tau_v_val in [20.0, 50.0, 100.0]:
    model = CANN1D_SFA(
        num=256,
        tau=1.0,
        tau_v=tau_v_val,
        k=8.1,
        a=0.3,
        A=0.2,
        J0=1.0,
        m=0.3
    )
    run_experiment(
        model,
        title=f"SFA时间常数: tau_v={tau_v_val}",
        save_path=None
    )

<SmoothTracking1D> Generating Task data: 500it [00:00, 987.72it/s] 


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 7448.12it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

<SmoothTracking1D> Generating Task data: 500it [00:00, 1604.13it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

**观察**：
- `tau_v=20.0`：快速适应，凸起可能漂移更快或领先于刺激
- `tau_v=50.0`：标准适应，中等漂移行为
- `tau_v=100.0`：缓慢适应，较弱的漂移效应

**关键洞察**：SFA 引入了预期动力学。适应电流产生的不对称性可以使凸起"预测"运动方向。这对路径积分模型很有用。

---

## 4. 后续步骤

恭喜您完成了所有基础教程！您现在已理解：
- 所有主要CANN参数及其物理含义
- 每个参数对隆起动力学的影响
- 如何进行系统的参数扫描实验
- 参数选择中涉及的权衡

### 参数选择指南

配置CANN模型用于您的应用时：

1. **从默认参数开始** - 默认参数提供稳定、行为良好的动力学
2. **将分辨率与需求匹配** - 更高的`num`用于精确空间编码，更低的用于提高效率
3. **调整时间动力学** - 根据所需响应速度调整`tau`
4. **平衡稳定性和灵活性** - `J0`和`k`控制吸引子强度
5. **匹配空间尺度** - 根据所需隆起宽度设置`a`

### 继续学习

从这里选择高级应用教程：

- **[教程5：分层路径积分网络](./05_hierarchical_network.ipynb)** - 学习具有多个空间尺度的分层路径积分
- **[教程6：Theta扫描系统模型](./06_theta_sweep_hd_grid.ipynb)** - 学习结合头向和网格细胞的Theta扫描系统
- **[教程7：Theta扫描位置细胞网络](./07_theta_sweep_place_cell.ipynb)** - 学习位置细胞网络和空间记忆

或探索其他场景：
- **场景2：数据分析** - 使用CANN分析实验神经数据
- **场景3：脑启发学习** - CANN的脑启发训练方法
- **场景4：端到端管道** - 从数据到发表的完整研究工作流程

### 关键要点

1. **参数相互关联** - 改变一个参数可能需要调整其他参数以保持稳定性
2. **可视化以理解** - 探索参数时始终使用能量景观可视化
3. **从简单开始** - 在转向SFA或2D变体之前从基础CANN1D开始
4. **记录您的选择** - 记下为什么选择特定参数值以便重现